## IMPORT DES LIBRAIRIES

In [1]:
# Package pour les expressions régulières
import re
# Package de manipaulation des tableaux et dataframe
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold, RFECV, SelectKBest, chi2, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,f1_score,classification_report,PrecisionRecallDisplay,precision_recall_curve,average_precision_score,precision_recall_fscore_support
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

from itertools import cycle
import pickle

# Boruta pour sélection de variables
from boruta import BorutaPy

# Package xgboost
from xgboost import XGBClassifier

# Package pour analyse statistique
import scipy.stats as ss
from scipy.stats import chi2_contingency

# Package pour représentation graphique
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import scikitplot as skplt

# Affichage Image (png, jpeg, html ...)
from IPython.display import Image, HTML

# Typing des fonctions
from typing import List, Optional

from functools import reduce

from sklearn.inspection import permutation_importance, PartialDependenceDisplay

import shap
from lime import lime_tabular

C:\Users\user\.conda\envs\SDA_P4\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Notebook de ciblage

In [22]:
# %load solutions.py
# ----------------------------------------------
# chargement des données
targeting_data = pd.read_csv("targeting_data.csv", sep = ",")

# Charger le model xgb_model.sav
#filename = "xgb_model_bef_cal.sav"
#filename = "bagging_model_bef_cal.sav"
filename = "logistic_model_bef_cal.sav"
xgb_model = pickle.load(open(filename, 'rb'))

# Prédiction sur le jeu de données test
y_pred = xgb_model.predict_proba(targeting_data)[:, 1]

# Créer un dataframe contenant l'identifiant de chaque individu et sa
# probabilité. Puis trier ce dataframe selon les probabilités prédites
prediction_df = (
    pd.DataFrame({"FIPS_code": targeting_data["FIPS_code"], "prediction": y_pred})
    .sort_values(["prediction"], ascending=False)
    .reset_index(drop=True)
)
print("prediction_df : ", prediction_df)
# Liste des individus à cibler
n = 1500
trageting_id = prediction_df.loc[0:(n-1), ["FIPS_code"]]
print("trageting_id : ", trageting_id)
# Chargez le jeu de données des labels réels
real_label = pd.read_csv("target_sample.csv", sep = ",")
print("real_label : ", real_label)
# Croiser les identifiants à cibler avec la table 
merge_pred_real_df = trageting_id.merge(
    real_label,
    how='inner',
    on='FIPS_code'
)

# Performance de notre modèle de score

# Nombre de cible dans les clients contacter
nb_campaign_target = sum(merge_pred_real_df["Party"])
# Nombre total de potentiels cibles
nb_real_target = sum(real_label["Party"])
# Part de la cible identifiée
perc_target_campaign = nb_campaign_target / nb_real_target

# Part des clients qu'un modèle aléatoire aurait touché
random_model_trageting = (
    round((n / targeting_data.shape[0]) * nb_real_target)
)


print(
    f"Parmi les {nb_real_target} county potentiels, {nb_campaign_target}, "
    f"soit {100 * round(perc_target_campaign, 4)} % de la cible ont été "
    f"identifiés en ciblant {n} county.\n"
    f"Un modèle aléatoire aurait identifié "
    f"{random_model_trageting} potentiels county.\n"
    f"Notre modèle de score fait donc {round(nb_campaign_target / random_model_trageting, 2)} "
    f"fois mieux qu'un modèle aléatoire."
)      

prediction_df :        FIPS_code    prediction
0         48301  9.999961e-01
1         38053  9.995010e-01
2         18087  9.991299e-01
3         48431  9.990744e-01
4         46057  9.990662e-01
...         ...           ...
3107      36081  2.353554e-13
3108       6073  1.170279e-14
3109       6059  5.410421e-15
3110      17031  7.392420e-19
3111       6037  2.928387e-46

[3112 rows x 2 columns]
trageting_id :        FIPS_code
0         48301
1         38053
2         18087
3         48431
4         46057
...         ...
1495      17195
1496      38003
1497      47045
1498      40041
1499       1015

[1500 rows x 1 columns]
real_label :        FIPS_code  Party
0          1001      1
1          1003      1
2          1005      1
3          1007      1
4          1009      1
...         ...    ...
3107      56037      1
3108      56039      0
3109      56041      1
3110      56043      1
3111      56045      1

[3112 rows x 2 columns]
Parmi les 2574 county potentiels, 1489, soit 57.85